# TPS-MAY22, Fast AI Model 🚀

Hello The purpose of this Notebook is to provide a detailed guide to develop a model using **fastai**
I just started the Course a few days ago...

**Objective:** Build a powerfull **fastai** model that can provide a competitive level.

**Strategy:** I think I will follow this strategy:

**Level 1+2 Fast AI Model**

**Loading the Data + Creating some Features.**
* Quick overview of the data to identify potential opportunities.
* Feature engineering using text information. (Massive boost in the score)

**FastAI Model Development**
* Define all the data processing steps requiered.
* Define the split technique that will be utilized.
* Define some of the parameters for the TabularDataLoader.
* Create a TabularDataLoader.
* Show one batch of data, to identify issues.
* Define some of the parameters of the tabular_learner.
* Define the tabular_learner
* Visualize the model architecture.
* Train the model for 1 epoch, this is kind of an initialization.
* Optimize the model learning rate.
* Train the model for multiple epochs.




**Level 3 Model Optimization**
* Work in Progress, Probably Hyperparam Optimization
* Improve the FastAI model becasue is kind of week at this stage

---
**Other Similar Implementations**
I been working on other architechtures at the same time, to see what works better

XGBoost and LGBM Models

https://www.kaggle.com/code/cv13j0/tps-may22-eda-gbdt

NN Model

https://www.kaggle.com/code/cv13j0/tps-may22-eda-neuronal-nets/edit/run/95032660

---

**Data Description**

For this challenge, you are given (simulated) manufacturing control data and are tasked to predict whether the machine is in state 0 or state 1. 
The data has various feature interactions that may be important in determining the machine state.

Good luck!

**Files**
* train.csv - the training data, which includes normalized continuous data and categorical data
* test.csv - the test set; your task is to predict binary target variable which represents the state of a manufacturing process
* sample_submission.csv - a sample submission file in the correct format

---
**Notebooks Ideas and Credits**

I took ideas or inspiration from the following notebooks, if you enjoy my work, please take a look to the notebooks that inspire my work.

TPSMAY22 Gradient-Boosting Quickstart: https://www.kaggle.com/code/ambrosm/tpsmay22-gradient-boosting-quickstart/notebook

Fast.AI: https://www.kaggle.com/code/venkatkumar001/fast-ai-1/notebook?scriptVersionId=94623621

kaggle TPS2022 May FastAI Baseline: https://www.kaggle.com/code/casati8/kaggle-tps2022-may-fastai-baseline/notebook



---

# 1.0 - Installing or Upgrading Libraries...
In this section I will install or upgrade any library that I'm not 100% sure Kaggle Kernels are keeping up to date, I can also use this section to install anything extra that i need

In [ ]:
%%capture
# Upgrade the fastai libraries, to use the latest updates.

!pip3 install --upgrade fastai

---

# 2.0 - Importing Libraries...
...

In [ ]:
%%time
# Load some of the key Python libraries for the model development.

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%time
# Load the fastai tabular modules.

import fastai; fastai.__version__
from fastai.tabular.all import *

---

# 3.0 - Notebook Configurations
...

In [ ]:
%%time
# I like to disable my Notebook Warnings.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
# Notebook Configuration.

# Amount of data we want to load into the Model.
DATA_ROWS = None
# Dataframe, the amount of rows and cols to visualize.
NROWS = 25
NCOLS = 15
# Main data location path...
BASE_PATH = '...'

In [ ]:
%%time
# Configure notebook display settings to only use 2 decimal places, tables look nicer.

pd.options.display.float_format = '{:,.5f}'.format
pd.set_option('display.max_columns', NCOLS) 
pd.set_option('display.max_rows', NROWS)

----

# 4.0 - Reading the Datasets
...

In [ ]:
%%time
# Load the train and test CSVs information into a Pandas DataFrame.

trn_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')
tst_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')

# Load the submission CSV information into a Pandas DataFrame.

sub = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

---

# 5.0 - Memory Optimization
In this section I will use the popular reduce memory function to optimize the memory consumption of the datatset, this is quite critical to not waste resources in the training stages of the model.

In [ ]:
%%time
# Creates a funtion for reduce the memory usage, this is a quite popular 
# function that has been around Kaggle for a while.

def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
%%time
# Here we invoque the memory reducion function.
# Apply it to the train dataset.

# trn_data = reduce_memory_usage(trn_data)

In [ ]:
%%time
# Here we invoque the memory reducion function.
# Apply it to the test dataset.

# tst_data = reduce_memory_usage(tst_data)

---

# 6.0 - Understanding the Information Loaded
...

In [ ]:
%%time
# Display the number of unique values for each variable, sorted by quantity.

trn_data.nunique().sort_values(ascending = True)

In [ ]:
%%time
# Display information about the variables.

trn_data.info()

---

# 7.0 - Feature Engineering
...

## 7.1 Create Features from a String of Characters

In [ ]:
%%time
# Define a function to exploit the information enbedded in a string of chars.

def count_chars(df, field):
    '''
    Describe something...
    '''
    
    for i in range(10):
        df[f'ch_{i}'] = df[field].str.get(i).apply(ord) - ord('A')
        
    df["unique_characters"] = df[field].apply(lambda s: len(set(s)))
    return df

In [ ]:
%%time
# Utilizes the new created funtion to generate more features.

trn_data = count_chars(trn_data, 'f_27')
tst_data = count_chars(tst_data, 'f_27')

---

## 7.2 Create Features Across Columns

In [ ]:
%%time
# Create new features across the dataset columns 

continuous_feat = ['f_00', 'f_01', 'f_02', 'f_03', 'f_04', 'f_05', 'f_06', 'f_19', 'f_20', 'f_21', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_28']

def stat_features(df, cols = continuous_feat):
    '''
    Calculate aggregated features across the selected continuous columns
    
    '''
    # Base statistical features.
    df['f_sum']  = df[continuous_feat].sum(axis=1)
    df['f_min']  = df[continuous_feat].min(axis=1)
    df['f_max']  = df[continuous_feat].max(axis=1)
    df['f_std']  = df[continuous_feat].std(axis=1)    
    df['f_mad']  = df[continuous_feat].mad(axis=1)
    df['f_mean'] = df[continuous_feat].mean(axis=1)
    df['f_kurt'] = df[continuous_feat].kurt(axis=1)

    # Extra statistical features more complex interactions.
    df['f_prod'] = df[continuous_feat].prod(axis=1)
    df['f_range'] = df[continuous_feat].max(axis=1) - df[continuous_feat].min(axis=1)
    df['f_count_pos']  = df[df[continuous_feat].gt(0)].count(axis=1)
    df['f_count_neg']  = df[df[continuous_feat].lt(0)].count(axis=1)

    return df

In [ ]:
%%time
# Utilizes the new created funtion to generate more features.

trn_data = stat_features(trn_data, continuous_feat)
tst_data = stat_features(tst_data, continuous_feat)

In [ ]:
%%time
# ...
trn_data.head()

In [ ]:
%%time
# ...
trn_data.isnull().sum().sum()

In [ ]:
%%time
def calculate_feat_int(df):
    df['i_02_21'] = (df.f_21 + df.f_02 > 5.2).astype(int) - (df.f_21 + df.f_02 < -5.3).astype(int)
    df['i_05_22'] = (df.f_22 + df.f_05 > 5.1).astype(int) - (df.f_22 + df.f_05 < -5.4).astype(int)
    i_00_01_26 = df.f_00 + df.f_01 + df.f_26
    df['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)
    return df

trn_data = calculate_feat_int(trn_data)
tst_data = calculate_feat_int(tst_data)

---

# 8.0 Data Processing, for Model Development
...

In [ ]:
%%time
# Create a list of Continuous features...

MAX_UNIQUE = 800_000
continuous_feat = [feat for feat in trn_data.columns if trn_data[feat].nunique() > MAX_UNIQUE]
continuous_feat.remove('id')
print(continuous_feat)

In [ ]:
%%time
# Create a list of Continuous features based on the Feature Eng. results, to avoid confusion...

add_continuous_feat = ['ch_0', 'ch_1', 'ch_2', 'ch_3', 'ch_4', 'ch_5', 'ch_6', 'ch_7', 'ch_8', 'ch_9', 'unique_characters', 
                       'f_sum', 'f_min', 'f_max', 'f_std', 'f_mad', 'f_mean', 'f_kurt', 'f_prod', 'f_range', 'f_count_pos', 'f_count_neg']

list_1 = set(continuous_feat)
list_2 = set(add_continuous_feat)
continuous_feat = list(list_1.union(list_2))

In [ ]:
%%time
# ...
print(continuous_feat)

In [ ]:
%%time
# Create a list of Categorical features...

categorical_feat = [feat for feat in trn_data.columns if feat not in continuous_feat]
categorical_feat.remove('target')
categorical_feat.remove('id')
categorical_feat.remove('f_27')
print(categorical_feat)

In [ ]:
%%time
# Display the Categorical features and the number of unique values...

trn_data[categorical_feat].nunique()

In [ ]:
%%time
# Overwrite the previous selection based on number of unique values..
# I'm trying to avoid the usage of embedding layers by the fastai learner..
# basically blanking the categorical_feat fields..

continuous_feat = ['unique_characters', 'f_06', 'ch_7', 'ch_0', 'ch_8', 'f_std', 'f_range', 'f_24', 'f_min', 'f_21', 'ch_2', 'f_03',
                   'f_sum', 'f_05', 'f_count_neg', 'f_22', 'f_02', 'ch_3', 'f_26', 'f_00', 'ch_6', 'f_23', 'f_mean',
                   'f_count_pos', 'ch_9', 'f_prod', 'f_kurt', 'ch_4', 'f_mad', 'f_max',
                   'f_25', 'f_04', 'f_20', 'f_19', 'f_01', 'f_28', 'ch_1', 'ch_5', 'f_07', 'f_08', 'f_09', 'f_10', 'f_11', 'f_12', 'f_13', 'f_14', 'f_15', 'f_16', 
                   'f_17', 'f_18', 'f_29', 'f_30']

categorical_feat = [] # Avoid the usage of embedding layers...

---

# 9.0 - Creating the fast.ai Model
In this step we will create the fast ai model and explain all the required steps...
Steps...
* Define all the data processing steps requiered.
* Define the split technique that will be utilized.
* Define some of the parameters for the TabularDataLoader.
* Create a TabularDataLoader.
* Show one batch of data, to identify issues.
* Define some of the parameters of the tabular_learner.
* Define the tabular_learner
* Visualize the model architecture.
* Train the model for 1 epoch, this is kind of an initialization.
* Optimize the model learning rate.
* Train the model for multiple epochs.
* Done...

In [ ]:
%%time
# Creating a list of the data preprocessing steps we want to apply to the model...

data_processing = [FillMissing, # Fill the missing values in continuous columns.
                   Categorify,  # Transform the categorical variables to something similar to pd.Categorical.
                   Normalize,   # Normalize the datset before trainig the model.
                  ]

In [ ]:
%%time
# Create a simple split methodology for the dataset to be used during training...
# Not used in this example...

split_sample = np.random.choice(trn_data.shape[0], 200) # ...

In [ ]:
%%time
# Create a dataloader to be used by the learner
# Difining some data loader params

batch_size = 1024
valid_pct  = 0.15

# Creting the tabular data loader
data = TabularDataLoaders.from_df(df = trn_data,                     # Here we pass tge train dataset.
                                  path = '.',                        # Here we pass the location path of the data, in this case null = '.'
                                  procs = data_processing,           # ...
                                  cat_names = categorical_feat,      # ...
                                  cont_names = continuous_feat,      # ...
                                  valid_pct = valid_pct,             # ...   
                                  bs = batch_size,                   # ... 
                                  y_block = CategoryBlock,           # ...
                                  y_names = 'target',                # ...
                                 )

In [ ]:
%%time
# Display a batch of the data loaded to the data loader that will be used in the training stage

data.show_batch()

In [ ]:
%%time
# Define the tabular learner to be used in the training stage
# Learner parameters

layers_definition = [1024, 256, 128, 128, 32] #...
emb_size = None #...
my_config = tabular_config(y_range = (0,1)) #...


# Learner definition
learn = tabular_learner(dls     = data,                #....
                        layers  = layers_definition,   #....
                        emb_szs = emb_size,            #....
                        metrics = [accuracy],          #....
                        config  = my_config,           #....
                       ).to_fp16()                     #....

In [ ]:
%%time
# Prints out the model architecture.
# As a summary for easy visualization.
learn.summary()

---

# 10.0 - Training the fast.ai Model
...

In [ ]:
%%time
# Fit one fastai cycle on the training dataset.
# It's like a warm start to identify the optimal LR for the model in the subsequent steps.

learn.fit_one_cycle(1)

In [ ]:
%%time
# Find the optimal Learning Rate for the model based on previous information...

learn.lr_find()

In [ ]:
%%time
# Fit 3 epochs on the training dataset.
# Using the LR identified in the previous step.

lr = 0.0005754399462603033
learn.fit_one_cycle(6, lr_max = lr)

In [ ]:
%%time
#...
learn.fine_tune(30, base_lr = lr, freeze_epochs = 3)

---

# 11.0 - Creating Model Predictions
...

In [ ]:
%%time
#...
dl = learn.dls.test_dl(tst_data)

In [ ]:
%%time
#...
dl.show_batch()

In [ ]:
%%time
#...
nn_predictions, _, preds = learn.get_preds(dl = dl, with_decoded = True)

In [ ]:
%%time
#...
nn_predictions, preds

In [ ]:
%%time
#...
sub.target = np.argmax(nn_predictions, axis = 1)

In [ ]:
%%time
# Submit predicitons from the CV trained Model...
sub.to_csv('submission_fastai.csv', index = None)

In [ ]:
%%time
# Submit predicitons from the CV trained Model...
sub.head()

---

---